In [1]:
# !pip install datasets
# !pip install evaluate
# ! pip install -U accelerate
# ! pip install -U transformers

In [1]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
from bs4 import BeautifulSoup
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import copy
from tqdm import tqdm
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

Setting ds_accelerator to cuda (auto detect)


In [2]:
id2label = {0: "NORMAL", 1: "TOXIC"}
label2id = {"NORMAL": 0, "TOXIC": 1}

In [3]:
#tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-xlarge-mnli")
#model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-xlarge-mnli", num_labels=2, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)
#model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-xlarge-mnli")

# Load model directly
# from transformers import AutoTokenizer, AutoModelForMaskedLM

# tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
# model = AutoModelForMaskedLM.from_pretrained("FacebookAI/xlm-roberta-base")

In [4]:
tokenizer = AutoTokenizer.from_pretrained("/home/sslashinin/kovakimyan/diploma/xlm-roberta-base/", local_files_only=True)

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("/home/sslashinin/kovakimyan/diploma/xlm-roberta-base/", local_files_only=True, ignore_mismatched_sizes=True)

Some weights of the model checkpoint at /home/sslashinin/kovakimyan/diploma/xlm-roberta-base/ were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at /home/sslashinin/kovakimyan/diploma/xlm-roberta-bas

In [6]:
dataset = load_dataset("textdetox/multilingual_toxicity_dataset")

dataset

Found cached dataset parquet (/home/sslashinin/.cache/huggingface/datasets/textdetox___parquet/textdetox--multilingual_toxicity_dataset-c4d45e458288ed14/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/9 [00:00<?, ?it/s]

DatasetDict({
    am: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    ar: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    de: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    en: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    es: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    hi: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    ru: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    uk: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
    zh: Dataset({
        features: ['text', 'toxic'],
        num_rows: 5000
    })
})

In [7]:
dataset["ru"]

Dataset({
    features: ['text', 'toxic'],
    num_rows: 5000
})

In [8]:
def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def cleanPunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def clear_sentance(sentance):
    sentance= re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = cleanPunc(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    stop_words = set(stopwords.words('english'))
    stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in  stopwords.words('english'))
    return sentance.strip()

In [9]:
preprocessed_text = []
for sentance in tqdm(dataset["ru"]['text']):
    preprocessed_text.append(clear_sentance(sentance))

  0%|          | 0/5000 [00:00<?, ?it/s]/tmp/job-1888907/ipykernel_265194/3189064709.py:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  sentance = BeautifulSoup(sentance, 'lxml').get_text()
100%|██████████| 5000/5000 [00:01<00:00, 2549.53it/s]


In [10]:
new_dataset = copy.deepcopy(dataset["ru"])

new_dataset = new_dataset.add_column("preprocessed_text", new_dataset['text'])

In [11]:
en_dataset = copy.deepcopy(new_dataset)

en_dataset_split = en_dataset.train_test_split(test_size=0.3)

en_dataset_split = en_dataset_split.rename_column("toxic", "label")

en_dataset_split["train"] = en_dataset_split["train"].remove_columns("text")
en_dataset_split["test"] = en_dataset_split["test"].remove_columns("text")

en_dataset_split

DatasetDict({
    train: Dataset({
        features: ['label', 'preprocessed_text'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['label', 'preprocessed_text'],
        num_rows: 1500
    })
})

In [12]:
def preprocess_function(examples):
    return tokenizer(examples["preprocessed_text"], truncation=True)

tokenized_dataset = en_dataset_split.map(preprocess_function, batched=True)

tokenized_dataset

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'preprocessed_text', 'input_ids', 'attention_mask'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['label', 'preprocessed_text', 'input_ids', 'attention_mask'],
        num_rows: 1500
    })
})

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
accuracy = evaluate.load("accuracy")

In [15]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [16]:
training_args = TrainingArguments(
    output_dir="deberta_xlarge_mnli_ft_ru",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    #save_strategy="epoch",
    #load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model()

/home/sslashinin/.conda/envs/kovakimyan/lib/python3.11/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ovakimyanchris. Use `wandb login --relogin` to force relogin


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.246700,0.952667
2,0.346600,0.241879,0.954667
3,0.188600,0.255611,0.960000
4,0.082000,0.228682,0.967333
5,0.038200,0.252315,0.968000


In [17]:
import torch

In [18]:
def run_model(text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    print(model.config.id2label[predicted_class_id])

In [20]:
text = "Ведёшь себя как дурвк"
run_model(text)

text = "Сегодня солнечный день"
run_model(text)

LABEL_1
LABEL_0


In [23]:
en_dataset_split["test"]

Dataset({
    features: ['label', 'preprocessed_text'],
    num_rows: 1500
})